In [2]:
from functools import partial

import astroplan as ap
import astropy.units as u
import numpy as np
import pandas as pd

from astropaul.database import database_connection
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2026-01-01", "2026-07-01")
proposal_cycle = "2026A"

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = f"LBT Proposal {proposal_cycle}"
with database_connection() as conn:
    creator = tlc.TargetListCreator(name=name, connection=conn)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
        tlc.ancillary_data_from_tess,
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        partial(tlc.filter_targets, criteria=lambda df: (df["Num PEPSI Observations"] > 0)),
        partial(tlc.add_observability, observing_session=session, time_resolution=2 * u.hour),
        partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
        partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
        partial(lbt.add_pepsi_params, **pepsi_args),
        partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
    ]
    tl = creator.calculate(verbose=False)

tl.target_list["PEPSI exp_time"] += 120 # add 2 minutes overhead to each target

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
# tl.target_list

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list = answer.target_list.assign(**new_cols)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list = answer.target_list.assign(**new_cols)
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:428: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

Name: LBT Proposal 2026A
Criteria:
  lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))
  lambda df: (df["Num PEPSI Observations"] > 0)
  Observability calculated at LBT in 2.0 h intervals
  AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
  lambda df: (df["Observable Any Night"])
  Inverse of: lambda df: df["Teff"].isna()
  lambda df: df["PEPSI exp_time"] < 600
  
22 targets:
    22 QuadEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    Count: (2, 0)
    Observable: (4, 546)
    PEPSI : (3, 6)
Associated tables:
     928 rows,  7 columns: Ephemerides
     181 rows, 13 columns: PEPSI Observations
     182 rows,  2 columns: Lunar Phases

111.5 minutes


In [12]:
tl.target_list

,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Target Source,Tic,Vmag,Teff,...,Observable Hours Min,PEPSI fiber,PEPSI cd_blue,PEPSI cd_blue_num_exp,PEPSI cd_red,PEPSI cd_red_num_exp,PEPSI snr,PEPSI exp_time,PEPSI priority,PEPSI notes
0,TIC 123098844,279.572833,44.698600,18:38:17.48,+44:41:54.96,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,123098844,11.136,6761.00,...,0.0,300,3,1,6,1,100,309,(see grid),
15,TIC 204698586,185.551833,-24.224844,12:22:12.44,-24:13:29.44,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,204698586,11.506,6392.00,...,0.0,300,3,1,6,1,100,393,(see grid),
25,TIC 25818450,352.743458,53.069150,23:30:58.43,+53:04:08.94,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,25818450,11.782,7172.00,...,0.0,300,3,1,6,1,100,461,(see grid),
26,TIC 260056937,63.922208,47.422197,04:15:41.33,+47:25:19.91,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,260056937,10.302,8030.00,...,0.0,300,3,1,6,1,100,201,(see grid),
35,TIC 278352276,307.503625,48.607056,20:30:00.87,+48:36:25.4,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,278352276,10.387,7156.00,...,0.0,300,3,1,6,1,100,214,(see grid),
36,TIC 283940788,8.851417,62.901594,00:35:24.34,+62:54:05.74,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,283940788,11.774,7384.00,...,0.0,300,3,1,6,1,100,448,(see grid),
40,TIC 286470992,45.330708,60.572294,03:01:19.37,+60:34:20.26,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,286470992,10.325,8693.00,...,0.0,300,3,1,6,1,100,201,(see grid),
44,TIC 307119043,14.827542,51.221642,00:59:18.61,+51:13:17.91,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,307119043,9.940,7815.00,...,0.0,300,3,1,6,1,100,181,(see grid),
49,TIC 317863971,110.567500,3.031925,07:22:16.2,+03:01:54.93,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,317863971,10.306,8506.00,...,0.0,300,3,1,6,1,100,201,(see grid),
52,TIC 322727163,309.716625,50.466819,20:38:51.99,+50:28:00.55,QuadEB,Kostov 2022 doi.org/10.3847/1538-4365/ac5458,322727163,10.997,7876.56,...,0.0,300,3,1,6,1,100,274,(see grid),


In [11]:
# make a CSV file for use in the PIT
column_map = [
    ("Name", "Target Name"), 
    ("RAJ2000", "RA"),
    ("DecJ2000", "Dec"),
    ("V", "Vmag"),
    ("pepsi_exposure_time", "PEPSI exp_time"),
]

proposal_table = pd.DataFrame()
for csv_name, tl_name in column_map:
    proposal_table[csv_name] = tl.target_list[tl_name]
proposal_table.to_csv(f"LBT {proposal_cycle} Proposal Targets.csv", index=False)

In [ ]:
foo = pd.read_csv(f"LBT {proposal_cycle} Proposal Targets.csv")
foo["hours"] = foo["pepsi_exposure_time"] / 3600
np.min(foo["pepsi_exposure_time"]) / 3600
foo.sort_values("Name")

,Name,RAJ2000,DecJ2000,pmRA,pmDec,V,pepsi_exposure_time,hours
38,BD +28 3402,293.751500,29.087250,31.298000,11.217000,8.880,180,0.050000
40,HD 102870,177.673833,1.764722,740.746000,-270.927000,3.600,180,0.050000
39,HD 112299,193.867875,25.738056,36.182000,-118.539000,8.380,180,0.050000
32,HD 18884,45.569875,4.089750,-10.410000,-76.850000,2.530,180,0.050000
35,HD 23169,55.971417,25.725417,67.904000,-87.136000,8.480,180,0.050000
34,HD 3765,10.205292,40.187167,356.318000,-669.205000,7.360,180,0.050000
36,HD 65583,120.133875,29.212361,-169.918000,-1164.932000,7.000,180,0.050000
37,HD 693,2.816083,-15.467972,-82.828000,-269.549000,4.890,180,0.050000
33,HD 92588,160.350792,-1.741500,-138.666000,-119.375000,6.260,180,0.050000
41,HIP 104548,317.660667,38.961167,4.399000,-13.309000,7.530,180,0.050000
